# Noised Datasets

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=1UtsutsiIQQXDEEsqjsQcjkwq4f5zLFBc', '/content/',quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1UtsutsiIQQXDEEsqjsQcjkwq4f5zLFBc
To: /content/DenoisedImages.zip
100%|██████████| 651M/651M [00:02<00:00, 220MB/s]


'/content/DenoisedImages.zip'

In [ ]:
!unzip -q /content/DenoisedImages.zip -d /content
!rm -rf /content/DenoisedImages.zip

replace /content/noised_15_sun/image_15.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
import os
noised_images = [f for f in os.listdir('/content/') if 'noised' in f]
noised_images

['noised_gray_25_sun',
 'noised_25_sun',
 'noised_50_sun',
 'noised_gray_15_sun',
 'noised_gray_50_sun',
 'noised_15_sun']

# Setup

In [ ]:
!git clone https://github.com/megvii-research/NAFNet
%cd NAFNet

!pip install -r requirements.txt
!python3 setup.py develop --no_cuda_ext

import gdown
gdown.download('https://drive.google.com/uc?id=1lsByk21Xw-6aW7epCwOQxvm6HYCQZPHZ', "/content/NAFNet/experiments/pretrained_models/", quiet=False)

Cloning into 'NAFNet'...
remote: Enumerating objects: 517, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 517 (delta 167), reused 132 (delta 132), pack-reused 321
Receiving objects: 100% (517/517), 16.19 MiB | 22.16 MiB/s, done.
Resolving deltas: 100% (276/276), done.
/content/NAFNet/NAFNet/NAFNet
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarnin

Downloading...
From: https://drive.google.com/uc?id=1lsByk21Xw-6aW7epCwOQxvm6HYCQZPHZ
To: /content/NAFNet/experiments/pretrained_models/NAFNet-SIDD-width32.pth
100%|██████████| 117M/117M [00:00<00:00, 117MB/s]


'/content/NAFNet/experiments/pretrained_models/NAFNet-SIDD-width32.pth'

In [ ]:
import torch
from basicsr.models import create_model
from basicsr.utils import img2tensor as _img2tensor, tensor2img, imwrite
from basicsr.utils.options import parse
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
%cd /content/NAFNet

/content/NAFNet


In [ ]:
opt_path = '/content/NAFNet/options/test/SIDD/NAFNet-width32.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False
NAFNet = create_model(opt)

 load net keys <built-in method keys of collections.OrderedDict object at 0x7e5ea80425c0>


In [ ]:
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

def img2tensor(img, bgr2rgb=False, float32=True):
    img = img.astype(np.float32) / 255.
    return _img2tensor(img, bgr2rgb=bgr2rgb, float32=float32)

def single_image_inference(model, img, save_path):
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      sr_img = tensor2img([visuals['result']])
      imwrite(sr_img, save_path)

# Test

In [ ]:
os.makedirs('/content/NAFNet_Results', exist_ok=True)

In [ ]:
for dataset in noised_images:
  print(f"Read {dataset}...")
  for img_name in tqdm(os.listdir(f'/content/{dataset}')):
    img_path = os.path.join(f'/content/{dataset}', img_name)
    img = imread(img_path)
    img = img2tensor(img)

    output_path = f'/content/NAFNet_Results/DENOISED_{dataset}'
    os.makedirs(output_path, exist_ok=True)
    out_path = os.path.join(output_path, img_name)
    single_image_inference(NAFNet, img, out_path)

Read noised_gray_25_sun...


100%|██████████| 80/80 [00:37<00:00,  2.12it/s]


Read noised_25_sun...


100%|██████████| 80/80 [00:37<00:00,  2.11it/s]


Read noised_50_sun...


...

100%|██████████| 80/80 [00:36<00:00,  2.16it/s]


Read noised_gray_50_sun...


100%|██████████| 80/80 [00:36<00:00,  2.16it/s]


Read noised_15_sun...


100%|██████████| 80/80 [00:37<00:00,  2.11it/s]


In [ ]:
noised_images

['noised_gray_25_sun',
 'noised_25_sun',
 'noised_50_sun',
 'noised_gray_15_sun',
 'noised_gray_50_sun',
 'noised_15_sun']

In [ ]:
%cd /content/NAFNet_Results

/content/NAFNet_Results


In [ ]:
for folder in os.listdir('/content/NAFNet_Results/DENOISED_noised_50_sun'):
  plt.imshow(plt.imread(f'/content/NAFNet_Results/DENOISED_noised_50_sun/{folder}'))
  plt.show()

In [ ]:
from google.colab import files

for folder in os.listdir('/content/NAFNet_Results'):
    folder_path = f'/content/NAFNet_Results/{folder}'
    zip_command = f'zip -r "{folder_path}.zip" "{folder_path}"'
    os.system(zip_command)
    files.download(f'/content/NAFNet_Results/{folder}.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>